In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')
from PIL import Image
import cv2
import pandas as pd

t = pd.read_csv('car_detection_dataset/train_bounding_boxes.csv')

N = len(t)
# Create a numpy array with all images
for i in range(N): 
    filename = 'car_detection_dataset/training_images/' + t['image'][i]
    image = np.array(Image.open(filename))
    image_col = image.ravel()[:,np.newaxis]
    if i==0:
        X_training = image_col
    else:
        X_training = np.hstack((X_training, image_col))

# Training feature matrices
X_train = X_training.T

t = t.drop('image', axis=1).round().to_numpy().astype(int)

training_set = np.hstack((X_train, t))
training_set.shape

(559, 770644)

In [2]:
images = pd.read_csv('car_detection_dataset/train_bounding_boxes.csv')['image']
#images

In [3]:
# 80/20 train-val split
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(training_set, shuffle=False, test_size=0.2)
X_train.shape, X_val.shape

((447, 770644), (112, 770644))

In [4]:
t_train = X_train[:,-4:]
t_val = X_val[:,-4:]
t_train.shape, t_val.shape

((447, 4), (112, 4))

In [5]:
X_train = X_train[:,:-4]
X_val = X_val[:,:-4]
X_train.shape, X_val.shape

((447, 770640), (112, 770640))

In [6]:
import tensorflow as tf
from tensorflow import keras
X_train_rs = tf.constant(X_train.reshape((X_train.shape[0],380,676,3)), dtype=tf.float32)
X_val_rs = tf.constant(X_val.reshape((X_val.shape[0],380,676,3)), dtype=tf.float32)

print('TF Training Object shape: ', X_train_rs.shape, '\nTF Validation Object shape: ', X_val_rs.shape)

2022-12-10 22:06:29.643336: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-10 22:06:30.165143: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79111 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:0f:00.0, compute capability: 8.0


TF Training Object shape:  (447, 380, 676, 3) 
TF Validation Object shape:  (112, 380, 676, 3)


In [7]:
transfer_layer=keras.applications.Xception(weights='imagenet', input_shape=(380, 676, 3), include_top=False)
transfer_layer.trainable = True
model = keras.models.Sequential([ 
    transfer_layer, 
    #keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.Dropout(rate=0.5),  
    keras.layers.Dense(50, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.Dropout(rate=0.5), 
    keras.layers.Dense(4, activation='relu') 
])
model.compile(optimizer=keras.optimizers.Adam(0.0003), loss=keras.losses.MeanSquaredError(), metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_rs, t_train, epochs=10, batch_size=32, 
                    validation_data=(X_val_rs, t_val), callbacks=[keras.callbacks.EarlyStopping(patience=5)])

In [8]:
#model.save('saved_models/regression.h5')
model = keras.models.load_model('saved_models/regression.h5')

In [ ]:
key_names = list(history.history.keys())
colors = ['-r','--b','-og','-.k']
plt.figure(figsize=(8,5))
plt.ylim(top=1.0)
for i in range(len(key_names)):
    plt.plot(history.history[key_names[i]], colors[i], label=key_names[i])
plt.legend(fontsize=15,ncol=2)
plt.title('Learning Curves', size=15);

In [ ]:
key_names = list(history.history.keys())
colors = ['-r','--b','-og','-.k']
plt.figure(figsize=(8,5))
for i in [0,2]:
    plt.plot(history.history[key_names[i]], colors[i], label=key_names[i])
plt.legend(fontsize=15,ncol=2)
plt.title('Learning Curves', size=15);

In [10]:
# Model's Predictions
y_train = model.predict(X_train_rs)
y_train = y_train.astype(int)
y_train.shape

2022-12-10 22:06:58.677356: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2022-12-10 22:06:59.790754: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Running ptxas --version returned 32512
2022-12-10 22:06:59.997223: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: ptxas exited with non-zero error code 32512, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-12-10 22:07:01.325540: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


(447, 4)